# Analysis of the language parameter



In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [63]:
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
sns.set_style("darkgrid")
from tqdm.notebook import tqdm, trange
from pandas.plotting import scatter_matrix

In [64]:
path = '../data/' 
hotels = pd.read_csv(path + '/features_hotels.csv')
data = pd.read_csv(path + 'data_language_utility.csv')

In [65]:
data = data.drop(['avatar_id','request_number'],axis=1)

In [66]:
data.loc[data.hotel_id == 334]

,city,date,language,mobile,hotel_id,price,stock
0,valletta,1,maltese,1,334,79,0
49,valletta,1,latvian,1,334,82,0
115,valletta,1,czech,1,334,80,0
157,valletta,1,estonian,1,334,78,0
1029,valletta,44,maltese,1,334,68,56
...,...,...,...,...,...,...,...
69256,valletta,8,luxembourgish,1,334,80,0
80720,valletta,9,maltese,1,334,79,0
80759,valletta,9,portuguese,1,334,82,0
80802,valletta,9,spanish,1,334,82,0


In [67]:
dic_lang = {'amsterdam':'dutch', 'copenhagen':'danish', 'madrid':'spanish', 'paris':'french', 'rome':'italian', 'sofia':'bulgarian', 'valletta':'maltese', 'vienna':'austrian' ,'vilnius':'lithuanian'}


In [68]:
# city = list(dic_lang)
# langu = [dic_lang[ci] for ci in city]
# df = pd.DataFrame([city,langu]).T
# df.rename({0:'city',1:"language_city"}, axis=1)
data.iloc[2]

city        valletta
date               1
language     maltese
mobile             1
hotel_id         732
price            176
stock              1
Name: 2, dtype: object

In [69]:
data['city_language'] = data['city'].map(dic_lang)
data['is_same_cl'] = data['city_language']==data['language']
# data['is_same_cl'] = False
# for i in range(len(data)):
#     data.iloc[i].is_same_cl = data.iloc[i].city_language == data.iloc[i].city


In [70]:
grouped = data.groupby(['city','date','mobile','hotel_id','is_same_cl']).agg({'price':['mean','var']})#mean()
grouped

price           
                                                 mean        var
city      date mobile hotel_id is_same_cl                       
amsterdam 2    1      9        False        86.666667   6.333333
                               True         84.000000        NaN
                      10       False       156.000000  21.000000
                               True        151.000000        NaN
                      12       False        80.666667   6.333333
...                                               ...        ...
vilnius   43   0      957      True         56.000000        NaN
                      974      False       106.666667  17.333333
                               True        104.000000        NaN
                      976      False        75.666667  10.333333
                               True         74.000000        NaN

[39840 rows x 2 columns]

In [71]:
grouped = grouped.reset_index()
grouped

city date mobile hotel_id is_same_cl       price           
                                                        mean        var
0      amsterdam    2      1        9      False   86.666667   6.333333
1      amsterdam    2      1        9       True   84.000000        NaN
2      amsterdam    2      1       10      False  156.000000  21.000000
3      amsterdam    2      1       10       True  151.000000        NaN
4      amsterdam    2      1       12      False   80.666667   6.333333
...          ...  ...    ...      ...        ...         ...        ...
39835    vilnius   43      0      957       True   56.000000        NaN
39836    vilnius   43      0      974      False  106.666667  17.333333
39837    vilnius   43      0      974       True  104.000000        NaN
39838    vilnius   43      0      976      False   75.666667  10.333333
39839    vilnius   43      0      976       True   74.000000        NaN

[39840 rows x 7 columns]

In [72]:
grouped.loc[grouped.is_same_cl == False].price['var']#.sort_values('var')

0         6.333333
2        21.000000
4         6.333333
6        32.333333
8        31.000000
           ...    
39830     4.333333
39832     9.333333
39834     4.333333
39836    17.333333
39838    10.333333
Name: var, Length: 19920, dtype: float64

In [73]:
grouped.loc[grouped.is_same_cl == False].loc[grouped.price['var'] < 1.].price['mean'] - grouped.iloc[grouped.loc[grouped.is_same_cl == False].loc[grouped.price['var'] < 1.].index+1].price['mean']

242     NaN
243     NaN
246     NaN
247     NaN
252     NaN
         ..
39639   NaN
39642   NaN
39643   NaN
39646   NaN
39647   NaN
Name: mean, Length: 2176, dtype: float64

In [74]:
grouped.iloc[grouped.loc[grouped.is_same_cl == False].loc[grouped.price['var'] < 1.].index+1]

city date mobile hotel_id is_same_cl price    
                                                  mean var
243    amsterdam    5      1        9       True  84.0 NaN
247    amsterdam    5      1       12       True  78.0 NaN
253    amsterdam    5      1       34       True  78.0 NaN
275    amsterdam    5      1      117       True  91.0 NaN
309    amsterdam    5      1      264       True  79.0 NaN
...          ...  ...    ...      ...        ...   ...  ..
39631    vilnius   42      0      914       True  54.0 NaN
39635    vilnius   42      0      950       True  71.0 NaN
39639    vilnius   42      0      955       True  54.0 NaN
39643    vilnius   42      0      957       True  56.0 NaN
39647    vilnius   42      0      976       True  74.0 NaN

[1088 rows x 7 columns]

In [75]:
grouped['sign_mean'] = - grouped.price['mean'] * grouped.is_same_cl + grouped.price['mean'] * (1-grouped.is_same_cl)

In [76]:
def percentage(x):
    """ alpha = 1.96 borne à 95% pour la loi normale  """
    moy = x.sum()
    return moy/x.max()

percentage.__name__ = 'percentage'

In [77]:
grouped['price_mean']=grouped.price['mean']
grouped['price_var']=grouped.price['var']
grouped.drop('price',axis=1,inplace=True)
grouped

/tmp/ipykernel_4747/819659357.py:3: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  grouped.drop('price',axis=1,inplace=True)


,city,date,mobile,hotel_id,is_same_cl,sign_mean,price_mean,price_var
,,,,,,,,
0,amsterdam,2,1,9,False,86.666667,86.666667,6.333333
1,amsterdam,2,1,9,True,-84.000000,84.000000,NaN
2,amsterdam,2,1,10,False,156.000000,156.000000,21.000000
3,amsterdam,2,1,10,True,-151.000000,151.000000,NaN
4,amsterdam,2,1,12,False,80.666667,80.666667,6.333333
...,...,...,...,...,...,...,...,...
39835,vilnius,43,0,957,True,-56.000000,56.000000,NaN
39836,vilnius,43,0,974,False,106.666667,106.666667,17.333333
39837,vilnius,43,0,974,True,-104.000000,104.000000,NaN


In [94]:
grouped = pd.read_csv('test.csv')

In [95]:
grouped

,Unnamed: 0,city,date,mobile,hotel_id,is_same_cl,sign_mean,price_mean,price_var
0,0,amsterdam,2,1,9,False,86.666667,86.666667,6.333333
1,1,amsterdam,2,1,9,True,-84.000000,84.000000,NaN
2,2,amsterdam,2,1,10,False,156.000000,156.000000,21.000000
3,3,amsterdam,2,1,10,True,-151.000000,151.000000,NaN
4,4,amsterdam,2,1,12,False,80.666667,80.666667,6.333333
...,...,...,...,...,...,...,...,...,...
39835,39835,vilnius,43,0,957,True,-56.000000,56.000000,NaN
39836,39836,vilnius,43,0,974,False,106.666667,106.666667,17.333333
39837,39837,vilnius,43,0,974,True,-104.000000,104.000000,NaN
39838,39838,vilnius,43,0,976,False,75.666667,75.666667,10.333333


In [100]:
df2 = grouped[['city','date','mobile','hotel_id','price_mean']].groupby(['city','date','mobile','hotel_id']).agg({'price_mean': [percentage]})
df2

price_mean
                               percentage
city      date mobile hotel_id           
amsterdam 2    1      9          1.969231
                      10         1.967949
                      12         1.966942
                      16         1.965265
                      22         1.966480
...                                   ...
vilnius   43   0      955        1.975904
                      956        1.969027
                      957        1.976744
                      974        1.975000
                      976        1.977974

[19920 rows x 1 columns]

In [107]:
print(f'percentage diff mean : {df2.price_mean.percentage.mean()}')
print(f'percentage diff var : {df2.price_mean.percentage.var()}')

percentage diff mean : 1.9711529825723841
percentage diff var : 0.0002941680171949086
